**Chat Model**


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)
chef_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)


chef_chain = chef_template | chat

In [5]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarain chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- Salt and pepper, to taste
- Fresh cilantro, for garnish
- Cooked rice or naan, for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and mix well to coat. Cover and marinate in the refrigerator for at least 1 hour, or overnight for best results.

2. P

AIMessageChunk(content="As a vegetarian chef, I can help you make a delicious vegetarian version of Chicken Tikka Masala by replacing the chicken with a suitable alternative. Here's how you can modify the recipe:\n\nIngredients:\n- 1 lb firm tofu, pressed and cut into bite-sized pieces OR 1 lb cauliflower florets\n- 1 cup plain yogurt (you can use dairy-free yogurt as a vegan alternative)\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon ginger, minced\n- 1 can (14 oz) tomato sauce\n- 1 tablespoon garam masala\n- 1 teaspoon ground cumin\n- 1 teaspoon ground coriander\n- 1 teaspoon paprika\n- 1/2 teaspoon turmeric\n- Salt and pepper, to taste\n- Fresh cilantro, for garnish\n- Cooked rice or naan, for serving\n\nInstructions:\n1. For tofu: Follow the same marinating process as with chicken, but be gentle when mixing to avoid breaking the tofu. For cauliflower: Skip the marinating step and proceed to the next.